# Path Explanations of Decision Tree
The purpose of this Notebook is to train a decision tree on a dataset and to extract the structure of the tree: the decisions at each node for each path and some statistics at each leaf. Training an accurate model is not the focus.

In [1]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import pandas as pd
import numpy as np

%matplotlib inline

/Users/gdeoliveira/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/gdeoliveira/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Load the Census Data data set

In [2]:
file_path = '/Users/gdeoliveira/Documents/DSS Projects/LIME/census_data.csv'

In [3]:
df = pd.read_csv(file_path)
df.drop(' fnlwgt', inplace=True, axis=1)
columns = df.columns.tolist()
df.columns = [x.strip() for x in columns]

print df.shape
print df.columns.tolist()

(32561, 14)
['age', 'workclass', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'target']


In [4]:
df.head(3)

,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [5]:
X = df.drop('target', axis=1)
y = df['target']
print X.shape
print y.shape

(32561, 13)
(32561,)


## Encode the target variable

In [6]:
le= LabelEncoder()
le.fit(y)
y = le.transform(y)
class_names = le.classes_
print y[:10]

[0 0 0 0 0 0 0 1 1 1]


## Encode the categorical features as integer values

In [7]:
categorical_features = X.select_dtypes(include=[object]).columns.tolist()
numerical_features = X.select_dtypes(exclude=[object]).columns.tolist()
categorical_feature_indices = [X.columns.tolist().index(i) for i in categorical_features]
print categorical_features
print categorical_feature_indices

['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
[1, 2, 4, 5, 6, 7, 8, 12]


In [8]:
# as you perform the encoding of each categorical column, store the encoding map in a dictionary for later use
label_encoder_map = {}
for feature, idx in zip(categorical_features, categorical_feature_indices):
    le = LabelEncoder()
    le.fit(X[feature])
    label_encoder_map[idx] = le.classes_.tolist()
    X[feature] = le.transform(X[feature])


In [9]:
print label_encoder_map[1]

[' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay']


In [10]:
X.head(3)

,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,7,9,13,4,1,1,4,1,2174,0,40,39
1,50,6,9,13,2,4,0,4,1,0,0,13,39
2,38,4,11,9,0,6,1,4,1,0,0,40,39


In [11]:
print X.shape

(32561, 13)


## One-Hot-Encode the Categorical Features

In [12]:
encoder = OneHotEncoder(sparse=False, categorical_features=categorical_feature_indices)
X_ohe = encoder.fit_transform(X)
print X_ohe.shape

(32561, 107)


In [13]:
ohe_columns = np.empty(shape=X_ohe.shape[1], dtype='object')
for enc, idx, col in zip(encoder.feature_indices_, categorical_feature_indices, categorical_features):
    values = label_encoder_map[idx] 
    for k, value in enumerate(values):
        ohe_columns[enc + k] = col + ':_' + value.strip()

ohe_columns[encoder.feature_indices_[-1]:] = numerical_features

In [14]:
X_ohe = pd.DataFrame(X_ohe, columns=ohe_columns)
print X_ohe.shape

(32561, 107)


In [15]:
X_ohe.head(3)

,workclass:_?,workclass:_Federal-gov,workclass:_Local-gov,workclass:_Never-worked,workclass:_Private,workclass:_Self-emp-inc,workclass:_Self-emp-not-inc,workclass:_State-gov,workclass:_Without-pay,education:_10th,...,native-country:_Thailand,native-country:_Trinadad&Tobago,native-country:_United-States,native-country:_Vietnam,native-country:_Yugoslavia,age,education_num,capital-gain,capital-loss,hours-per-week
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,39.0,13.0,2174.0,0.0,40.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,50.0,13.0,0.0,0.0,13.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,38.0,9.0,0.0,0.0,40.0


## Train/Test Split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_ohe, y, random_state=0)

In [17]:
print X_train.shape
print X_test.shape
print X_test.shape[0]/float(X.shape[0])

(24420, 107)
(8141, 107)
0.250023033691


## Train and fit Decision Tree using Grid Search

In [18]:
param_grid = {'max_depth':[6, 8, 10, 12],
              'min_samples_leaf':[1, 10, 20, 40, 60, 80]
             }
dtc = DecisionTreeClassifier(random_state=0)

grid_search_cv = GridSearchCV(estimator = dtc, 
                              param_grid=param_grid, 
                              cv=5, 
                              scoring='precision')

grid_search_cv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [6, 8, 10, 12], 'min_samples_leaf': [1, 10, 20, 40, 60, 80]},
       pre_dispatch='2*n_jobs', refit=True, scoring='precision', verbose=0)

In [19]:
dtc_best = grid_search_cv.best_estimator_
print 'max_depth =', dtc_best.get_params()['max_depth']
print 'min_samples_leaf =', dtc_best.get_params()['min_samples_leaf']
dtc_best.fit(X_train, y_train)

max_depth = 6
min_samples_leaf = 60


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=60,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

In [20]:
y_pred = dtc_best.predict(X_test)
print 'accuracy =', metrics.accuracy_score(y_test, y_pred)
print 'recall =', metrics.recall_score(y_test, y_pred)
print 'precision =', metrics.precision_score(y_test, y_pred)

accuracy = 0.852352290873
recall = 0.529767911201
precision = 0.795454545455


## Decision Tree Feature Importance

In [21]:
importances = dtc_best.feature_importances_
indices = np.argsort(importances)[::-1]

feature_importance = []
for f in range(X_train.shape[1]):
    feature_importance.append([X_train.columns[indices[f]], importances[indices[f]]])

df_i = pd.DataFrame(feature_importance, columns=['feature', 'weight'])
df_i.head(10)

,feature,weight
0,marital_status:_Married-civ-spouse,0.449833
1,education_num,0.229923
2,capital-gain,0.213031
3,age,0.045695
4,capital-loss,0.036000
5,hours-per-week,0.023229
6,relationship:_Wife,0.002227
7,workclass:_Private,0.000061
8,occupation:_Farming-fishing,0.000000
9,marital_status:_Married-spouse-absent,0.000000


# Extract Decision Tree Paths on Test Data Set
 
http://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html

In [22]:
# The decision estimator has an attribute called tree_  which stores the entire
# tree structure and allows access to low level attributes. The binary tree
# tree_ is represented as a number of parallel arrays. The i-th element of each
# array holds information about the node `i`. Node 0 is the tree's root. NOTE:
# Some of the arrays only apply to either leaves or split nodes, resp. In this
# case the values of nodes of the other type are arbitrary!
#
# Among those arrays, we have:
#   - left_child, id of the left child of the node
#   - right_child, id of the right child of the node
#   - feature, feature used for splitting the node
#   - threshold, threshold value at the node
#

# Using those arrays, we can parse the tree structure:

n_nodes = dtc_best.tree_.node_count
children_left = dtc_best.tree_.children_left
children_right = dtc_best.tree_.children_right
feature = dtc_best.tree_.feature
threshold = dtc_best.tree_.threshold

feature_names = [X_test.columns[k] for k in feature]

In [23]:
# The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth

while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print 'number of nodes = ', n_nodes
print 'number of leaves = ', sum(is_leaves)

number of nodes =  69
number of leaves =  35


In [24]:
# gather the decisions at each node
node_decisions = {}
for i in range(n_nodes):
    if is_leaves[i]:
        text = "leaf node."
        node_decisions[i] = text
    else:
        text = ("test node: %s <= %s then %s else %s."
              % (feature_names[i], threshold[i], children_left[i], children_right[i],))
        node_decisions[(i, children_left[i])] = '%s <= %s' % (feature_names[i], threshold[i])
        node_decisions[(i, children_right[i])] = '%s > %s' % (feature_names[i], threshold[i])
    

In [25]:
# get the decision path for each record in X_test
node_indicator = dtc_best.decision_path(X_test)


In [26]:
# this is the decision path for the 0th record
record = 0
node_indicator[record,:].indices

array([0, 1, 2, 3, 4, 8, 9], dtype=int32)

In [27]:
# find the leaves for each record in X_test
leave_ids = dtc_best.apply(X_test)

In [28]:
# create dictionary that stores all records per leaf
records_in_leaves = {k: [] for k in set(leave_ids)}
for i, leave_id in enumerate(leave_ids):
    records_in_leaves[leave_id].append(i)

In [29]:
# compute target aggreations in each leaf
target_percentages_per_leaf = {}
for leave_id, records in records_in_leaves.iteritems():
    target_sum = sum([y_test[x] for x in records])
    record_count = len(records)
    target_percentages_per_leaf[leave_id] = [target_sum, record_count, float(target_sum)/record_count]

## Print path info for a given record

In [30]:
# this function prints out the decisions at each node, 
# the leaf_id and the target percentages for a given record
def print_path_info(record):
    nodes_traversed = node_indicator[record, :].indices
    links = [(i,j) for (i,j) in zip(nodes_traversed[:-1], nodes_traversed[1:])]
    for link in links:
        print node_decisions[link]

    print '\nleaf_id =', leave_ids[record]
    print 'target percentages =', target_percentages_per_leaf[leave_ids[record]]

In [31]:
# print path out for record 0
print_path_info(0)

marital_status:_Married-civ-spouse <= 0.5
capital-gain <= 7073.5
education_num <= 12.5
capital-loss <= 2103.0
hours-per-week > 40.5
age <= 38.5

leaf_id = 9
target percentages = [19, 382, 0.049738219895287955]


## Store the paths in a pandas DataFrame

In [32]:
def get_path_info(record):
    nodes_traversed = node_indicator[record, :].indices
    links = [(i,j) for (i,j) in zip(nodes_traversed[:-1], nodes_traversed[1:])]
    path_decisions = ''
    for link in links:
        path_decisions += ', ' + node_decisions[link]
    path_decisions = [path_decisions[2:]]
    counts = target_percentages_per_leaf[leave_ids[record]]
    path_decisions.extend(counts)
    return path_decisions


In [33]:
data = []
for leave_id, records in records_in_leaves.iteritems():
    # take the first record to follow the path to that leave_id
    record = records[0]
    path_result = get_path_info(record)
    data.append(path_result)


In [34]:
result = pd.DataFrame(data, columns = ['node_decisions', 'target_count', 'row_count', 'target_fraction'])

In [35]:
result.head()

,node_decisions,target_count,row_count,target_fraction
0,"marital_status:_Married-civ-spouse <= 0.5, cap...",5,1662,0.003008
1,"marital_status:_Married-civ-spouse <= 0.5, cap...",38,1170,0.032479
2,"marital_status:_Married-civ-spouse <= 0.5, cap...",19,382,0.049738
3,"marital_status:_Married-civ-spouse <= 0.5, cap...",22,210,0.104762
4,"marital_status:_Married-civ-spouse <= 0.5, cap...",3,10,0.300000


In [36]:
print result.shape

(35, 4)
